In [1]:
import pandas as pd
from pulp import *

# Define Nodes
hubs = ['CVG', 'AFW']
focus_cities = ['Leipzig', 'Hyderabad', 'San_Bernardino']

# Capacities
hub_capacity = {'CVG': 95650, 'AFW': 44350}
focus_capacity = {'Leipzig': 85000, 'Hyderabad': 19000, 'San_Bernardino': 36000}

# Center Demand
center_demand = {
    'Paris': 6500, 'Cologne': 640, 'Hanover': 180, 'Bengaluru': 9100, 
    'Coimbatore': 570, 'Delhi': 19000, 'Mumbai': 14800, 'Cagliari': 90, 
    'Catania': 185, 'Milan': 800, 'Rome': 1700, 'Katowice': 170, 
    'Barcelona': 2800, 'Madrid': 3700, 'Castle Donington': 30, 'London': 6700, 
    'Mobile': 190, 'Anchorage': 175, 'Fairbanks': 38, 'Phoenix': 2400, 
    'Los Angeles': 7200, 'Ontario': 100, 'Riverside': 1200, 'Sacramento': 1100, 
    'San Francisco': 1900, 'Stockton': 240, 'Denver': 1500, 'Hartford': 540, 
    'Miami': 3400, 'Lakeland': 185, 'Tampa': 1600, 'Atlanta': 3000, 
    'Honolulu': 500, 'Kahului/Maui': 16, 'Kona': 63, 'Chicago': 5100, 
    'Rockford': 172, 'Fort Wayne': 200, 'South Bend': 173, 'Des Moines': 300, 
    'Wichita': 290, 'New Orleans': 550, 'Baltimore': 1300, 'Minneapolis': 1700, 
    'Kansas City': 975, 'St. Louis': 1200, 'Omaha': 480, 'Manchester': 100, 
    'Albuquerque': 450, 'New York': 11200, 'Charlotte': 900, 'Toledo': 290, 
    'Wilmington': 150, 'Portland': 1200, 'Allentown': 420, 'Pittsburgh': 1000, 
    'San Juan': 1100, 'Nashville': 650, 'Austin': 975, 'Dallas': 3300, 
    'Houston': 3300, 'San Antonio': 1100, 'Richmond': 600, 'Seattle/Tacoma': 2000, 
    'Spokane': 260
}
centers = list(center_demand.keys())

In [2]:
# Cost Processing 
costs = {}

def add_cost(orig, dest, cost):
    if cost is not None: costs[(orig, dest)] = cost

# Hub -> Focus Costs
add_cost('CVG', 'Leipzig', 1.5)
add_cost('CVG', 'San_Bernardino', 0.5)
add_cost('AFW', 'San_Bernardino', 0.5)

# Raw Data for Hub/Focus -> Center
raw_cost_data = [
    ('Paris', 1.6, None, 0.5, 1.1, None), ('Cologne', 1.5, None, 0.5, 1, None),
    ('Hanover', 1.5, None, 0.5, 1, None), ('Bengaluru', None, None, 1.5, 0.5, None),
    ('Coimbatore', None, None, 1.5, 0.5, None), ('Delhi', None, None, 1.5, 0.5, None),
    ('Mumbai', None, None, 1.5, 0.5, None), ('Cagliari', 1.5, None, 0.5, 1, None),
    ('Catania', 1.5, None, 0.5, 1, None), ('Milan', 1.5, None, 0.5, 1, None),
    ('Rome', 1.5, None, 0.5, 1.1, None), ('Katowice', 1.4, None, 0.5, 1, None),
    ('Barcelona', 1.5, None, 0.5, 1, None), ('Madrid', 1.6, None, 0.5, 1.1, None),
    ('Castle Donington', 1.4, None, 0.5, None, None), ('London', 1.6, None, 0.75, 1.1, None),
    ('Mobile', 0.5, 0.5, None, None, 0.5), ('Anchorage', 1.3, 1, None, None, 0.7),
    ('Fairbanks', 1.4, 1, None, None, 0.7), ('Phoenix', 0.5, 0.5, None, None, 0.5),
    ('Los Angeles', 0.5, 0.5, None, None, None), ('Ontario', 0.5, 0.5, None, None, None),
    ('Riverside', 0.5, 0.5, None, None, None), ('Sacramento', 0.5, 0.5, None, None, 0.5),
    ('San Francisco', 0.5, 0.5, None, None, 0.5), ('Stockton', 0.5, 0.5, None, None, 0.5),
    ('Denver', 0.5, 0.5, None, None, 0.5), ('Hartford', 0.5, 0.5, 1.5, None, 0.5),
    ('Miami', 0.5, 0.5, None, None, 0.7), ('Lakeland', 0.5, 0.5, None, None, 0.7),
    ('Tampa', 0.5, 0.5, None, None, 0.7), ('Atlanta', 0.5, 0.5, None, None, 0.6),
    ('Honolulu', None, 0.5, None, None, 0.5), ('Kahului/Maui', None, 0.5, None, None, 0.5),
    ('Kona', None, 0.5, None, None, 0.5), ('Chicago', 0.5, 0.5, None, None, 0.5),
    ('Rockford', 0.5, 0.5, None, None, 0.5), ('Fort Wayne', 0.5, 0.5, None, None, 0.5),
    ('South Bend', 0.5, 0.5, None, None, 0.5), ('Des Moines', 0.5, 0.5, None, None, 0.5),
    ('Wichita', 0.5, 0.5, None, None, 0.5), ('New Orleans', 0.5, 0.5, None, None, 0.5),
    ('Baltimore', 0.5, 0.5, 1.5, None, 0.7), ('Minneapolis', 0.5, 0.5, None, None, 0.5),
    ('Kansas City', 0.5, 0.5, None, None, 0.5), ('St. Louis', 0.5, 0.5, None, None, 0.5),
    ('Omaha', 0.5, 0.5, None, None, 0.5), ('Manchester', 0.5, 0.5, 1.5, None, 0.7),
    ('Albuquerque', 0.5, 0.5, None, None, 0.5), ('New York', 0.5, 0.5, 1.6, None, 0.7),
    ('Charlotte', 0.5, 0.5, None, None, 0.7), ('Toledo', 0.5, 0.5, None, None, 0.5),
    ('Wilmington', 0.5, 0.5, None, None, 0.7), ('Portland', 0.5, 0.5, None, None, 0.5),
    ('Allentown', 0.5, 0.5, 1.5, None, 0.7), ('Pittsburgh', 0.5, 0.5, None, None, 0.6),
    ('San Juan', 0.5, 0.5, None, None, 1), ('Nashville', 0.5, 0.5, None, None, 0.5),
    ('Austin', 0.5, 0.25, None, None, 0.5), ('Dallas', 0.5, None, None, None, 0.5),
    ('Houston', 0.5, 0.25, None, None, 0.5), ('San Antonio', 0.5, 0.25, None, None, 0.5),
    ('Richmond', 0.5, 0.5, None, None, 0.7), ('Seattle/Tacoma', 0.5, 0.5, None, None, 0.5),
    ('Spokane', 0.5, 0.5, None, None, 0.5)
]

for row in raw_cost_data:
    add_cost('CVG', row[0], row[1]); add_cost('AFW', row[0], row[2])
    add_cost('Leipzig', row[0], row[3]); add_cost('Hyderabad', row[0], row[4])
    add_cost('San_Bernardino', row[0], row[5])

In [3]:
# Model Definition
prob = LpProblem("Amazon_Distribution", LpMinimize)

# Decision Variables
x = LpVariable.dicts("x_Hub_to_Focus", ((i, j) for i in hubs for j in focus_cities if (i, j) in costs), lowBound=0, cat='Continuous')
y = LpVariable.dicts("y_Hub_to_Center", ((i, k) for i in hubs for k in centers if (i, k) in costs), lowBound=0, cat='Continuous')
z = LpVariable.dicts("z_Focus_to_Center", ((j, k) for j in focus_cities for k in centers if (j, k) in costs), lowBound=0, cat='Continuous')

# Objective Function
prob += (
    lpSum(x[i, j] * costs[i, j] for i, j in x) + 
    lpSum(y[i, k] * costs[i, k] for i, k in y) + 
    lpSum(z[j, k] * costs[j, k] for j, k in z)
), "Total_Transportation_Cost"

In [4]:
# Constraints

# Hub Capacity
for i in hubs:
    outflow = lpSum(x[i, j] for j in focus_cities if (i, j) in costs) + \
              lpSum(y[i, k] for k in centers if (i, k) in costs)
    prob += outflow <= hub_capacity[i], f"Hub_Capacity_{i}"

# Focus City Inbound Capacity
for j in focus_cities:
    inflow = lpSum(x[i, j] for i in hubs if (i, j) in costs)
    prob += inflow <= focus_capacity[j], f"Focus_Inbound_Capacity_{j}"

# Flow Conservation
for j in focus_cities:
    inflow = lpSum(x[i, j] for i in hubs if (i, j) in costs)
    outflow = lpSum(z[j, k] for k in centers if (j, k) in costs)
    prob += inflow == outflow, f"Flow_Conservation_{j}"

# Center Demand
for k in centers:
    inflow = lpSum(y[i, k] for i in hubs if (i, k) in costs) + \
             lpSum(z[j, k] for j in focus_cities if (j, k) in costs)
    prob += inflow == center_demand[k], f"Center_Demand_{k}"

In [6]:
# Solve/Output
prob.solve()

print(f"Status: {LpStatus[prob.status]}")
print(f"Total Minimized Cost: ${value(prob.objective):,.2f}")

Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Sep  3 2024 

command line - cbc /var/folders/gm/42rg51qs64v1llshp9cr0m5w0000gn/T/a0869d45863d454688c6c49c8f5e8f06-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/gm/42rg51qs64v1llshp9cr0m5w0000gn/T/a0869d45863d454688c6c49c8f5e8f06-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 78 COLUMNS
At line 655 RHS
At line 729 BOUNDS
At line 730 ENDATA
Problem MODEL has 73 rows, 191 columns and 385 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 47 (-26) rows, 144 (-47) columns and 290 (-95) elements
Perturbing problem by 0.001% of 1.6 - largest nonzero change 9.0370431e-05 ( 0.018965159%) - largest zero change 4.8550215e-05
0  Obj 150023.19 Primal inf 81777.101 (44)
48  Obj 199392.93 Primal inf 138.2 (4)
52  Obj 199480.94
Optimal - objective value 199476.25
After Postsolve, objective 199476.25, infeasibilities - dual

In [ ]:
print("\n DETAILED CONSTRAINT CHECK ")

# 1. Check Hubs
print("\n[Hub Capacity]")
for i in hubs:
    used = sum(x[i, j].varValue for j in focus_cities if (i, j) in costs) + \
           sum(y[i, k].varValue for k in centers if (i, k) in costs)
    print(f"Hub {i}: Used {used:,.0f} / {hub_capacity[i]:,.0f}")

# 2. Check Focus Cities
print("\n[Focus City Conservation]")
for j in focus_cities:
    inflow = sum(x[i, j].varValue for i in hubs if (i, j) in costs)
    outflow = sum(z[j, k].varValue for k in centers if (j, k) in costs)
    print(f"{j}: In {inflow:,.0f} == Out {outflow:,.0f}")

# 3. Check Demand 
print("\n[Center Demand Satisfaction ]")
for k in centers[:10]:
    received = sum(y[i, k].varValue for i in hubs if (i, k) in costs) + \
               sum(z[j, k].varValue for j in focus_cities if (j, k) in costs)
    print(f"{k}: Received {received:,.0f} / Required {center_demand[k]}")